In [ ]:
import ast
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
# Newer versions of sklearn
from sklearn.model_selection import train_test_split

In [ ]:
pedestrians_df = pd.read_csv('pedestrian_df.csv')
for col_name in ['bounding_boxes', 'frame_numbers', 'moving slow', 'stopped', 'handwave', 'look', 'clear path', 'crossing', 'moving fast', 'looking', 'standing', 'slow down', 'nod', 'speed up']:
    pedestrians_df[col_name] = pedestrians_df[col_name].apply(ast.literal_eval)
pedestrians_df.head()

In [ ]:
count = 0
for i, row in pedestrians_df.iterrows():
    count += len(row['frame_numbers'])
    
print("Number of Pedestrian-Frames: %d" % count)

In [ ]:
# Let's take a more in-depth look at that first row:
print(pedestrians_df.iloc[0])
print(pedestrians_df.iloc[0]['frame_numbers'])

In [ ]:
#Dropping unrelated columns
pedestrians_df=pedestrians_df.drop(['ped_ind','bounding_boxes'], axis=1)

In [ ]:
#split data 70-30
from sklearn.model_selection import train_test_split
X = pedestrians_df.iloc[:,:-1]
y= pedestrians_df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Explode training datasets
X_train = X_train.reset_index()
X_test = X_test.reset_index()
X_train= X_train.apply(pd.Series.explode)
X_test= X_test.apply(pd.Series.explode)


In [ ]:
#Encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for i in X_train.columns:
        if i != 'video_id':
            X_train[i]=le.fit_transform(X_train[i])
    
for j in X_test.columns:
    if j != 'video_id':
        X_test[j]=le.fit_transform(X_test[j])

In [ ]:
# Feature Analysis
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 6))
sns.heatmap(X_train.corr(), annot=True)


In [ ]:
#X_train and X_test split (target feature= crossing)

y_train_1 = X_train.iloc[:,8]
X_train_1 = X_train.drop('crossing', axis=1)
X_train_1 = X_train_1.iloc[:, 3:]
y_test_1 = X_test.iloc[:,8]
X_test_sub = X_test.drop('crossing', axis=1)
X_test_1 = X_test_sub.iloc[:, 3:]

In [ ]:
#Random Forest Algorithm
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train_1, y_train_1)

In [ ]:
# Accuracy check
from sklearn.metrics import accuracy_score

y_pred_1=rfc.predict(X_test_1)
print("Accuracy:",accuracy_score(y_test_1, y_pred_1)*100)

In [ ]:
# Important Features

import matplotlib.pyplot as plt
import numpy as np

importance= rfc.feature_importances_
plt.title('Feature Importance')
sorted_indices = np.argsort(importance)[::-1]
plt.bar(range(X_train_1.shape[1]), importance[sorted_indices], align='center')
plt.xticks(range(X_train_1.shape[1]), X_train_1.columns[sorted_indices], rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Create prediction dataframe with respective video_id

pred= pd.DataFrame(X_test_sub['video_id'])
col=pd.DataFrame(y_pred_1, columns = ['val'])
pred=pred.join(col)
len(pred)

In [ ]:
# Calculate Final crossing result based on per frame bool values

cal=[]
final=[]
for k in range(max(pred.index)+1):
    count0=0
    count1=0
    
    cal.append(pred.loc[k]['val'])
    for f in cal[0]:
        if f == 0:
            count0+=1
        else:
            count1+=1
    
    if count0>count1:
        final.append(0)
        cal.clear()
        print('if executed',k)
    else:
        final.append(1)
        cal.clear()
        print('else executed',k)
        

In [ ]:
# Overall crossing accuracy

print("Final Accuracy:",accuracy_score(y_test, final)*100)